# Setup

In [97]:
import os
import json

import s3fs

from s3path import S3Path

import duckdb
import polars as pl

from dotenv import load_dotenv

## Environment variables

In [2]:
load_dotenv()

True

## PostgreSQL configurations

In [127]:
pg_user = os.getenv("PGUSER")
pg_password = os.getenv("PGPASSWORD")
pg_host = os.getenv("PGHOST", "localhost")
pg_port = os.getenv("PGPORT", "5432")
pg_database = os.getenv("PGDATABASE")

pg_conn_str = f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}"

## MinIO configurations

In [4]:
s3_endpoint = "http://localhost:9000"
s3_region = "us-east-1"
s3_access_key_id = os.getenv("MINIO_ROOT_USER", "admin")
s3_secret_access_key = os.getenv("MINIO_ROOT_PASSWORD", "admin")

# Analytics and time series

## pg_mooncake

The storage layer of `pg_mooncake` is Delta Lake, so we should be able to access the tables externally.

### Retrieve storage path

In [134]:
table_path = pl.read_database_uri(
    query="""
        SELECT path
        FROM mooncake.columnstore_tables
        WHERE table_name = 'youtube'
    """,
    uri=pg_conn_str,
).item()

table_path

's3://lakehouse/mooncake_datalabtech_youtube_102460/'

### Load using DuckDB

After setting up the secret to connect to MinIO, we attempt to count the number of rows in the `youtube` table.

In [135]:
con = duckdb.connect()

con.install_extension("httpfs")
con.load_extension("httpfs")

con.execute(
    f"""
    CREATE SECRET (
      TYPE S3,
      KEY_ID '{s3_access_key_id}',
      SECRET '{s3_secret_access_key}',
      ENDPOINT '{s3_endpoint.lstrip("http://")}',
      REGION '{s3_region}',
      USE_SSL false,
      URL_STYLE 'path'
    )
    """
)

result = con.execute('SELECT count(*) FROM delta_scan(?)', (table_path,)).fetchone()
print(result)

con.close()

(1541128,)


## timescaledb_toolkit

In [137]:
pl.read_database_uri(query="SELECT count(*) FROM youtube_ts", uri=pg_conn_str)

count
i64
0
